<a href="https://colab.research.google.com/github/DianaRoor/-/blob/main/%D0%9E%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D1%8C_%D0%BA%D0%BB%D0%B0%D1%81%D1%81%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%86%D0%B8%D0%B8_%D0%B4%D0%BB%D1%8F_%D1%86%D0%B2%D0%B5%D1%82%D0%BA%D0%BE%D0%B2_Iris'%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Домашнее задание
Обучите модель классификации для цветков Iris'а

Примерная последовательность действий:

Взять данные - https://drive.google.com/file/d/18ksAxTxBkp15LToEg46BHhwp3sPIoeUU/view?usp=sharing
Загрузить в pyspark
При помощи VectorAssembler преобразовать все колонки с признаками в одну (использовать PipeLine - опционально)
Разбить данные на train и test
Создать модель линейной регресии или модель дерева и обучить ее
Воспользоваться MulticlassClassificationEvaluator для оценки качества на train и test множестве

In [1]:
!apt-get update

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [945 kB]
Get

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark

In [3]:
!ls

sample_data  spark-3.5.1-bin-hadoop3  spark-3.5.1-bin-hadoop3.tgz


In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

In [5]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [6]:
df = spark.read.csv('iris.CSV', inferSchema=True, header=True)

In [7]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [12]:
df.describe().show(5)

+-------+------------------+-------------------+------------------+------------------+---------+------------------+
|summary|      sepal_length|        sepal_width|      petal_length|       petal_width|  variety|       variety_num|
+-------+------------------+-------------------+------------------+------------------+---------+------------------+
|  count|               150|                150|               150|               150|      150|               150|
|   mean| 5.843333333333335|  3.057333333333334|3.7580000000000027| 1.199333333333334|     NULL|               1.0|
| stddev|0.8280661279778637|0.43586628493669793|1.7652982332594662|0.7622376689603467|     NULL|0.8192319205190406|
|    min|               4.3|                2.0|               1.0|               0.1|   Setosa|                 0|
|    max|               7.9|                4.4|               6.9|               2.5|Virginica|                 2|
+-------+------------------+-------------------+------------------+-----

In [9]:
df.show()

+------------+-----------+------------+-----------+-------+-----------+
|sepal_length|sepal_width|petal_length|petal_width|variety|variety_num|
+------------+-----------+------------+-----------+-------+-----------+
|         5.1|        3.5|         1.4|        0.2| Setosa|          0|
|         4.9|        3.0|         1.4|        0.2| Setosa|          0|
|         4.7|        3.2|         1.3|        0.2| Setosa|          0|
|         4.6|        3.1|         1.5|        0.2| Setosa|          0|
|         5.0|        3.6|         1.4|        0.2| Setosa|          0|
|         5.4|        3.9|         1.7|        0.4| Setosa|          0|
|         4.6|        3.4|         1.4|        0.3| Setosa|          0|
|         5.0|        3.4|         1.5|        0.2| Setosa|          0|
|         4.4|        2.9|         1.4|        0.2| Setosa|          0|
|         4.9|        3.1|         1.5|        0.1| Setosa|          0|
|         5.4|        3.7|         1.5|        0.2| Setosa|     

In [11]:
df.columns

['sepal_length',
 'sepal_width',
 'petal_length',
 'petal_width',
 'variety',
 'variety_num']

In [17]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

In [88]:
from pyspark.ml import Pipeline

indexer = StringIndexer(inputCol='variety', outputCol='label')
assembler = VectorAssembler(inputCols=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], outputCol='features')
pipeline = Pipeline(stages=[indexer, assembler])

In [89]:
pipelineTrained = pipeline.fit(df)

In [90]:
pipelineTrained.transform(df).show()

+------------+-----------+------------+-----------+-------+-----------+-----+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|variety|variety_num|label|         features|
+------------+-----------+------------+-----------+-------+-----------+-----+-----------------+
|         5.1|        3.5|         1.4|        0.2| Setosa|          0|  0.0|[5.1,3.5,1.4,0.2]|
|         4.9|        3.0|         1.4|        0.2| Setosa|          0|  0.0|[4.9,3.0,1.4,0.2]|
|         4.7|        3.2|         1.3|        0.2| Setosa|          0|  0.0|[4.7,3.2,1.3,0.2]|
|         4.6|        3.1|         1.5|        0.2| Setosa|          0|  0.0|[4.6,3.1,1.5,0.2]|
|         5.0|        3.6|         1.4|        0.2| Setosa|          0|  0.0|[5.0,3.6,1.4,0.2]|
|         5.4|        3.9|         1.7|        0.4| Setosa|          0|  0.0|[5.4,3.9,1.7,0.4]|
|         4.6|        3.4|         1.4|        0.3| Setosa|          0|  0.0|[4.6,3.4,1.4,0.3]|
|         5.0|        3.4|         1.5| 

## Разобьем данные на данные для обучения и проверки

In [98]:
from pyspark.ml.classification import DecisionTreeClassifier

In [95]:
train, test = transformed_df.randomSplit([0.8, 0.2], seed=12345)

In [106]:
classifier = DecisionTreeClassifier(featuresCol='features', labelCol='label')
model = classifier.fit(train)

In [108]:
classifier = classifier.fit(train)

In [109]:
train_classifier_res=classifier.transform(train)
test_classifier_res=classifier.transform(test)

In [110]:
train_classifier_res.show()

+------------+-----------+------------+-----------+----------+-----------+-----+-----------------+--------------+-------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|   variety|variety_num|label|         features| rawPrediction|  probability|prediction|
+------------+-----------+------------+-----------+----------+-----------+-----+-----------------+--------------+-------------+----------+
|         4.3|        3.0|         1.1|        0.1|    Setosa|          0|  0.0|[4.3,3.0,1.1,0.1]|[43.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|         4.4|        2.9|         1.4|        0.2|    Setosa|          0|  0.0|[4.4,2.9,1.4,0.2]|[43.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|         4.4|        3.0|         1.3|        0.2|    Setosa|          0|  0.0|[4.4,3.0,1.3,0.2]|[43.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|         4.4|        3.2|         1.3|        0.2|    Setosa|          0|  0.0|[4.4,3.2,1.3,0.2]|[43.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|         4.5|        2.3| 

In [114]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [116]:
evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')


In [118]:
evaluator.evaluate(train_classifier_res)

0.9922480620155039

In [119]:
evaluator.evaluate(test_classifier_res)

1.0